### 1. 사람인 사이트에 들어가 직무를 입력한다.
### 2. 검색 결과에서 신입만 본다
### 3. 검색 결과에서 기본적으로 일반, 파견, 대행이 모두 눌려있음 -> 일반만 눌려있게 한다.
### 4. 일반 눌린 결과에서 5페이지까지 내용 긁어오기 ('기업명', '제목', '주소', '링크')
### 5. 긁어온 결과 csv file로 저장하기


In [ ]:
import time
import warnings
warnings.filterwarnings('ignore')
    # 경고무시
import pandas as pd
from tqdm import tqdm_notebook
import chromedriver_autoinstaller

from bs4 import BeautifulSoup as bs
from selenium import webdriver
import requests

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import getpass
from urllib.request import Request, urlopen

In [ ]:
# By.CSS_SELECTOR, #txtSource

# 1. 사람인 사이트에 들어가 직무를 입력한다.
# 2. 검색 결과에서 신입만 본다
# 3. 검색 결과에서 기본적으로 일반, 파견, 대행이 모두 눌려있음 -> 일반만 눌려있게 한다.

driver = webdriver.Chrome()
driver.get("https://www.saramin.co.kr/zf_user/")
question = input("검색할 직무를 입력하세요 : ")
time.sleep(2)
#검색창에 직무 입력후 확인
driver.find_element(By.CSS_SELECTOR, "#btn_search").click()
driver.find_element(By.CSS_SELECTOR, "#ipt_keyword_recruit").send_keys(question)
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, "#btn_search_recruit").click()
time.sleep(2)


#경력 -> 신입 -> 닫기 -> 선택된 00건 검색

driver.find_element(By.CSS_SELECTOR, "#sp_main_wrapper > div.default_option > div.search_option.career_option > button").click()
time.sleep(1)
driver.find_element(By.CSS_SELECTOR, "#sp_main_wrapper > div.default_option > div.search_option.career_option.open > div > div.area_check > div:nth-child(1)").click()
time.sleep(1)
driver.find_element(By.CSS_SELECTOR, "#sp_main_wrapper > div.default_option > div.search_option.career_option.open > div > div.area_btn > button.closeDefaultOptionLayer.btn_basic_type02.btn_close").click()
driver.find_element(By.CSS_SELECTOR, "#search_btn").click()

time.sleep(2)
# 채용정보 더보기 클릭
    # 검색 수가 적어 채용정보가 없는 경우도 있으니 except로 pass
try:
    driver.find_element(By.CSS_SELECTOR,".view_more.track_event").click()
    time.sleep(1)
    #일반, 파견, 대행이 모두 눌려있음 -> 나는 일반만 보고싶으니까 파견, 대행을 클릭해서 없애준다.
    driver.find_element(By.CSS_SELECTOR,"#recruit_info > div.header > div > div:nth-child(2) > div").click()
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR,"#recruit_info > div.header > div > div:nth-child(3) > div").click()
    
except:
    pass




In [ ]:
## 4. 일반 눌린 결과에서 5페이지까지 내용 긁어오기 ()
## 5. 긁어온 결과 csv file로 저장하기

title_t = []
company_t = []
addr_t = []
link_t = []

# 5페이지까지 크롤링
# url을 동적으로 첨부하고 싶었지만... 계속되는 오류로 붙여넣기를 진행했다.

for i in range(1,6):
    url = "https://www.saramin.co.kr/zf_user/search/recruit?searchType=search&searchword=%EB%8D%B0%EC%9D%B4%ED%84%B0&company_cd=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7&exp_cd=1&panel_type=&search_optional_item=y&search_done=y&panel_count=y&preview=y&recruitPage="+str(i)+"&recruitSort=relation&recruitPageCount=40&inner_com_type=&show_applied=&quick_apply=&except_read=&ai_head_hunting=&mainSearch=n"
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
    request = Request(url, headers = header)
    response = urlopen(request)
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')

    
    # 제목
    title = soup.select('#recruit_info_list > div.content > div > div.area_job > h2 > a > span')
    for t in title:
        title_t.append(t.text)
        
    # 회사 이름

    company = soup.select('#recruit_info_list > div.content > div > div.area_corp > strong > a')
    for t in range(len(company)):
        a = company[t].text.strip()
        companys = a.replace(",","").replace("\n", "")
        company_t.append(companys)
    
    
    #회사 위치
    addr = soup.select('#recruit_info_list > div.content > div > div.area_job >div.job_condition > span:nth-child(1)')
    for t in addr:
        addr_t.append(t.text)
    
    # 공고 링크
        
    link = soup.select('#recruit_info_list > div.content > div > div.area_job > h2 > a') 
    #'https://www.saramin.co.kr' + link[0]['href']

    for l in link:
        link_t.append('https://www.saramin.co.kr' + l['href'])
        

col = ['기업명', '제목', '주소', '링크']
data = pd.DataFrame(list(zip(title_t,company_t,addr_t,link_t)), columns=col)
data.to_csv('saramin.csv', index =False, encoding='utf-8-sig')
print("크롤링완료")